In [ ]:
# Generic imports
import pandas as pd
import plotly

# 5G visualization logic
import trace_plotting

In [ ]:
# Output from kubectl get --raw /apis/metrics.k8s.io/v1beta1/pods/. Each line a JSON output
filename = 'traces/test106/test106_kpidump.log'

# Wireshark trace of NAS messages
wireshark_traces = [
    'traces/test106/test106_tcpdump_AMF_N1N2.pcap',
    'traces/test106/test106_tcpdump_SMF_N4.pcap',
    'traces/test106/test106_tcpdump_SMF_SBI.pcap'
]

In [ ]:
data_to_plot = trace_plotting.parse_k8s_kpis_as_dataframe(filename)
display(data_to_plot)

In [ ]:
# Import Wireshark capture
packets_df = trace_plotting.import_pcap_as_dataframe(
    wireshark_traces, 
    http2_ports = "32445,5002,5000,32665,80,32077,5006,8080,3000,8081",
    wireshark_version = '3.4.3')
display(packets_df)
wireshark_imported = packets_df is not None
one_wireshark_trace = len(packets_df.file.unique()) == 1

In [ ]:
# Placeholder to check some values
# pd.set_option('display.max_rows', None)
# df[(df['namespace']=='amf') & (df['timestamp']=='2021-01-29 09:59:06+00:00')]
display(packets_df)
# display(packets_df.summary.unique())

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

# Just an example how to generate the title
title = 'Test {0}'.format(re.search(r'[\d]+', filename).group(0))
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
#display(data_to_plot)

# fig = go.Figure()
# fig = make_subplots(specs=[[{"secondary_y": True}]])
if not wireshark_imported:
    fig = make_subplots(
        rows=3, 
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05)
else:
    fig = make_subplots(
        rows=4, 
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.2, 0.2, 0.2, 0.4])

for idx,namespace in enumerate(data_to_plot['namespace'].unique()):
    series = data_to_plot[data_to_plot['namespace']==namespace]
    data_text = namespace + ', ' + series['timestamp'].apply(trace_plotting.datetime_to_str)
    series_color = colors[idx%len(colors)]
    for idx, column in enumerate(['cpu', 'memory', 'pod count']):
        fig.add_trace(trace_plotting.generate_scatterplot_for_k8s_kpis(series, namespace, idx==0, data_text, series_color, column),
                      row=idx+1, col=1)
    
if wireshark_imported:
    for wireshark_plot in trace_plotting.generate_scatterplots_for_wireshark_traces(packets_df, filter_column='file'):
        fig.add_trace(wireshark_plot, row=4, col=1)

fig.update_traces(
    marker_size=4,
    hoverlabel_align = 'right')
fig.update_yaxes(title_text='CPU [vCPU]', row=1, col=1)
fig.update_yaxes(title_text='Memory [GB]', row=2, col=1)
fig.update_yaxes(title_text='Pod count', row=3, col=1)
if not wireshark_imported:
    fig['layout'].update(height=700)
    fig.update_xaxes(title_text='Date', row=3, col=1)
else:
    fig['layout'].update(height=1000)
    fig.update_xaxes(title_text='Date', row=4, col=1)
# fig.update_layout(hovermode='x unified')
fig.update_layout(title_text=title)

if wireshark_imported:
    # Add color layout to procedure plot
    fig.update_layout(shapes=trace_plotting.get_protocol_shapes(packets_df))

fig.show()
fig.write_html("{0}.html".format(title))